In [1595]:
import numpy as np
import scipy as scp
import math
import time

# Sistemas Lineares com Matrizes Simétricas


### a) Escreva um código que monte a matriz A para n = 1000.

In [1596]:
def create_band_matrix(size):
    main_diagonal_value, secondary_diagonal_value, tertiary_diagonal_value = 100, -4, 1 
    band_width = 5
    distance_one_diagonal_from_main_diagonal, distance_two_diagonals_from_main_diagonal = 1, 2

    matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(max(0, i - band_width), min(size, i + band_width + 1)):
            if i == j:
                matrix[i, j] = main_diagonal_value

            elif abs(i - j) == distance_one_diagonal_from_main_diagonal:
                matrix[i, j] = secondary_diagonal_value

            elif abs(i - j) == distance_two_diagonals_from_main_diagonal:
                matrix[i, j] = tertiary_diagonal_value

    return matrix



In [1597]:
size = 1000
BandMatrix = create_band_matrix(size)

### b) Escreva um código que faça a decomposição de Cholesky de uma matriz simétrica definida positiva qualquer.




In [1598]:
def cholesky_decomposition(A):
    size = len(A)
    L = np.zeros((size, size))

    for i in range(size):
        for j in range(i + 1):
            if i == j:
                L[i, j] = (A[i, i] - np.sum(L[i, :j] ** 2)) ** (0.5)
            else:
                L[i, j] = (A[i, j] - np.sum(L[i, :j] * L[j, :j])) / L[j, j]

    return L


In [1599]:
A = np.array([[2, -1, 0],
              [-1, 2, -1],
              [0, -1, 2]])

L = cholesky_decomposition(A)

L

array([[ 1.41421356,  0.        ,  0.        ],
       [-0.70710678,  1.22474487,  0.        ],
       [ 0.        , -0.81649658,  1.15470054]])

### c) Usando as rotinas implementadas anteriormente, escreva um código para resolver um sistema $Ax=b$.

In [1600]:
def solve_lower_triangular(A, b, size):
  x = np.zeros(size)
  size = len(b)

  x[0] = b[0] / A[0, 0]
  for i in range(1, size):
      x[i] = b[i]
      for j in range(i - 1, -1, -1):
          x[i] = x[i] - (A[i, j] * x[j])
      x[i] = x[i] / A[i, i]

  return x

def solve_upper_triangular(A, b, size):
    x = np.zeros(size)
    size = len(b)

    x[size-1] = b[size-1] / A[size-1, size-1]
    for i in range(size-2, -1, -1):
        x[i] = b[i]
        for j in range(i+1, size):
            x[i] = x[i] - x[j] * A[i, j]
        x[i] = x[i] / A[i, i]

    return x


In [1601]:
def cholesky_LS(size):
  A = BandMatrix
  b = np.array([1/size**4]*size)

  H = cholesky_decomposition(A)

  y = solve_lower_triangular(H, b, size)
  x = solve_upper_triangular(H.T, y, size)

  return x

In [1602]:
size = 1000
x = cholesky_LS(size)
b = np.array([1/size**4]*size)
A = BandMatrix

Ax = A@x
residualError = abs(Ax - b)
print("Erro:\n", np.linalg.norm(residualError))

Erro:
 3.293678593616497e-27


### d) Escreva um código implementando o método de Jacobi para resolver um sistema $Ax=b$. Utilize o erro absoluto como critério de parada.

In [1603]:
def gauss_jacobi_solver(matrix, vector, tolerance):
    size = len(vector)

    x_previous = np.array([0] * size)
    x_current = np.array([np.inf] * size)

    diagonal = np.diag(np.diag(matrix))
    off_diagonal = np.eye(size) - np.linalg.solve(diagonal, matrix)
    constants = np.linalg.solve(diagonal, vector)

    iterations = 0
    while np.linalg.norm(x_current - x_previous) > tolerance:
        iterations += 1
        x_previous = x_current
        x_current = off_diagonal @ x_previous + constants

    return x_current, iterations



In [1604]:
# variáveis de controle
n = 1000
error = 10**(-15)

# a resposta possui dois campos, o primeiro é os valores do sistema linear e o segundo são as
# iterações necessárias para o erro da sequência ficar menor que o erro estabelecido.
x, it = gauss_jacobi_solver(BandMatrix,  np.array([1/size**4]*size), error)

print(f'Sistema resolvido com {it} iterações, com erro abaixo de {error}.')

Sistema resolvido com 1 iterações, com erro abaixo de 1e-15.


/var/folders/zs/jv6vwbhn4sz069dlcp9ktj1w0000gn/T/ipykernel_47211/2890091414.py:15: RuntimeWarning: invalid value encountered in matmul
  x_current = off_diagonal @ x_previous + constants


### e) Vamos comparar o método direto de Cholesky com o interativo de Jacobi neste exemplo. Observe quanto tempo leva para resolver o sistema usando Cholesky. Quantas iterações foram necessárias no método de Jacobi para obtermos a mesma precisão da solução dada pelo método de Cholesky ?

In [1605]:
n = 1000
before = time.time()
x = cholesky_LS(n)
after = time.time()

print('Cholesky:')
print(f'Time: {after-before} s.')
print(f'Precision: {abs(np.linalg.norm(np.linalg.solve(A, b) - x))}.')

Cholesky:
Time: 1.3477301597595215 s.
Precision: 8.044834298464815e-30.


In [1606]:
precision = abs(np.linalg.norm(np.linalg.solve(A, b) - x))

before = time.time()
ans, it = gauss_jacobi_solver(BandMatrix, np.array([1/size**4]*size), precision)
after = time.time()

print('Jacobi: ')
print(f'Time: {after - before} s.')
print(f'Iterations: {it}.')

Jacobi: 
Time: 0.04159712791442871 s.
Iterations: 1.


/var/folders/zs/jv6vwbhn4sz069dlcp9ktj1w0000gn/T/ipykernel_47211/2890091414.py:15: RuntimeWarning: invalid value encountered in matmul
  x_current = off_diagonal @ x_previous + constants


### f) É possível melhorar a implementação da decomposição de Cholesky para o exemplo em questão ?

Dado que a matriz A é uma matriz pentadiagonal, podemos usar as propriedade dessa matriz para otimizar o código original. Para isso podemos ignorar quaisquer valores que são 0 na nossa matriz, ou seja, o método de Cholesky só precisa ser feito para a diagonal principal e as duas diagonais acima e abaixo dela. Dessa forma o número de iterações necessárias para completar o algoritmo é reduzido. Abaixo está a implementação dessa otimização, assim como o tempo médio que ela demora para calcular o mesmo sistema linear calculado pelo método de Cholesky original anteriormente:

In [1607]:
def modified_cholesky_decomposition(matrix):
    n = np.shape(matrix)[0]
    decomposed_matrix = np.zeros_like(matrix)

    for k in range(n):
        start_band_row = max(0, k - 4)
        end_band_row = min(n, k + 5)
        diagonal_element = matrix[k, k]
        diagonal_sum = np.sum(decomposed_matrix[k, start_band_row:k] ** 2)
        decomposed_matrix[k, k] = math.sqrt(diagonal_element - diagonal_sum)

        for i in range(k + 1, end_band_row):
            start_band_col = max(0, i - 4)
            off_diagonal_element = matrix[i, k]
            row_elements = decomposed_matrix[i, start_band_col:k]
            column_elements = decomposed_matrix[k, start_band_col:k]
            column_sum = np.sum(row_elements * column_elements)
            decomposed_matrix[i, k] = (off_diagonal_element - column_sum) / decomposed_matrix[k, k]

    return decomposed_matrix


In [1608]:
startTimeCholeskyModified = time.time()
matrix = modified_cholesky_decomposition(BandMatrix)
endTimeCholeskyModified = time.time()

print("New Time:", endTimeCholeskyModified - startTimeCholeskyModified)

New Time: 0.015152215957641602


# Método de Newton Para Sistemas Não-*Lineares*

## 1) Observando a equação, vê-se a necessidade de calcular a matriz inverdsa da Jacobiana. É possível calcular inversas usando a decomposição LU, sendo assim, implemente um código que:

### a) Calcule a fatoração LU de uma matriz qualquer

In [1609]:
def lu_decomposition(matrix):
    size = np.shape(matrix)[0]
    lower_triangular = np.eye(size)
    upper_triangular = np.zeros((size, size))

    for current_row in np.arange(size):
        for current_column in np.arange(current_row, size):
            current_element = matrix[current_row, current_column]
            for prev_column in np.arange(current_row):
                current_element -= lower_triangular[current_row, prev_column] * upper_triangular[prev_column, current_column]
            upper_triangular[current_row, current_column] = current_element
        
        for next_row in np.arange(current_row + 1, size):
            current_element = matrix[next_row, current_row]
            for prev_column in np.arange(current_row):
                current_element -= lower_triangular[next_row, prev_column] * upper_triangular[prev_column, current_row]
            lower_triangular[next_row, current_row] = current_element / upper_triangular[current_row, current_row]

    return lower_triangular, upper_triangular


In [1610]:
A = np.array([[1, 2,0], [1, 3,1],[-2, 0,1]])
L, U = lu_decomposition(A)

print("L @ U:")
print(L@U)
print("\nMatriz original:")
print(A)

L @ U:
[[ 1.  2.  0.]
 [ 1.  3.  1.]
 [-2.  0.  1.]]

Matriz original:
[[ 1  2  0]
 [ 1  3  1]
 [-2  0  1]]


### b) Resolva um sistema linear cujo lado direito é uma matriz.

In [1611]:
def solve_linear_system_lu(A, b):
  L, U = lu_decomposition(A)
  y = solve_lower_triangular(L, b, len(b))
  x = solve_upper_triangular(U, y, len(b))

  return x

In [1612]:
# função que resolve um sistema linear em que o lado direito é uma matriz
def solve_linear_system_lu_matrix(A, B):
  X = []
  for b in B.T:
    X.append(solve_linear_system_lu(A, b).tolist())
  
  return np.array(X)
b = np.array([3, 5, -1])
B = np.array([[ 3, 3, 6],
              [ 5, 4, 2],
              [-1, 1, 1]])
X = solve_linear_system_lu_matrix(A, B)
print(X)

[[ 1.          1.          1.        ]
 [-1.          2.         -1.        ]
 [-5.33333333  5.66666667 -9.66666667]]


### c) Finalmente, calcula a inversa de uma matriz qualquer resolvendo vários sistemas lineares.

In [1613]:
def inverse_matrix(matrix):
    n = len(matrix)
    identity = np.identity(n)
    lower_triangular, upper_triangular = lu_decomposition(matrix)
    inverse_matrix = np.zeros((n, n))

    for col in range(n):
        y = np.zeros(n)
        for row in range(n):
            y[row] = identity[row][col] - sum(lower_triangular[row][j] * y[j] for j in range(row))
        
        x = np.zeros(n)
        for row in range(n-1, -1, -1):
            x[row] = (y[row] - sum(upper_triangular[row][j] * x[j] for j in range(row+1, n))) / upper_triangular[row][row]

        inverse_matrix[:, col] = x

    return inverse_matrix


In [1614]:
originalMatrix = np.array([[ 3, 3, 6],
              [ 5, 4, 2],
              [-1, 1, 1]])

inv = inverse_matrix(originalMatrix)
print("Original Matrix:\n", (originalMatrix))
print("Inverse Matrix:\n", (inv))

Original Matrix:
 [[ 3  3  6]
 [ 5  4  2]
 [-1  1  1]]
Inverse Matrix:
 [[ 0.05128205  0.07692308 -0.46153846]
 [-0.17948718  0.23076923  0.61538462]
 [ 0.23076923 -0.15384615 -0.07692308]]


## 2) Implemente o método de Newton para sistemas usando a rotina implementada no item anterior. Lembre-se de especificar o critério de parada utilizado.

In [1615]:
def solve_newton_system_with_absolute_error(Function, Jacobian, initial_guess, error):
    x = np.array(initial_guess)
    while np.linalg.norm(Function(x)) > error:
        x = x - np.dot(inverse_matrix(Jacobian(x)), Function(x))

    return x

In [1616]:
def Function(x):
  fun = np.array([
      x[0]**2 - 2*x[0] - x[1] + 1,
      x[0]**2 + x[1]**2 - 1,
  ])
  return fun

In [1617]:
def Jacobian(x):
  jac = np.array([
      [2*x[0] - 2 , -1    ],
      [2*x[0]     , 2*x[1]]
  ])
  return jac

In [1618]:
x0 = solve_newton_system_with_absolute_error(Function, Jacobian, [-1, -1], 10e-10)
x1 = solve_newton_system_with_absolute_error(Function, Jacobian, [-1, 1], 10e-10)

print(f'Soluçoes encontradas: {x0, x1}')

Soluçoes encontradas: (array([ 1.00000000e+00, -2.36781787e-16]), array([-1.88477779e-11,  1.00000000e+00]))


## 3) Implemente novamente o método de Newton para sistemas usando o algoritmo anterior. Utilize as rotinas já implementadas neste trabalho para resolução do sistema linear, indicando sempre qual está usando.

In [1619]:
def newton_new_algorithm(F, Jacobian, initial, error):
  x = np.array(initial)
  while np.linalg.norm(F(x)) > error:
    z = solve_linear_system_lu(Jacobian(x), F(x))
    x = x - z

  return x
  
x0 = solve_newton_system_with_absolute_error(Function, Jacobian, [-1, -1], 10e-10)
x1 = solve_newton_system_with_absolute_error(Function, Jacobian, [-1, 1], 10e-10)
print(f'First, Second: {np.round(x0), np.round(x1)}')

First, Second: (array([ 1., -0.]), array([-0.,  1.]))


Como se pode ver, ambas as soluções são as mesmas que a do exercício anterior, portanto o algoritmo está funcionando

## 4) Dada a equação de um círculo $(x-a)^2 + (y-b)^2 = R^2$ e três pontos que passam por esse circulo:
|Coordenada X|Coordenada Y|
| :-------: | :-------:  |
|$x_1 = 8.21$ | $y_1 = 0.00$ |
|$x_2 = 0.34$ | $y_2 = 6.62$ |
|$x_3 = 5.96$ | $y_3 = -1.12$|

### a) Monte um sistema não-linear para determinar $a,b,R$.

In [1620]:
def Function(x):
    f = np.zeros(3, dtype=np.float64)
    f[0] = (8.21 - x[0]) ** 2 + (0.00 - x[1]) ** 2 - x[2] ** 2
    f[1] = (0.34 - x[0]) ** 2 + (6.62 - x[1]) ** 2 - x[2] ** 2
    f[2] = (5.96 - x[0]) ** 2 + (-1.12 - x[1]) ** 2 - x[2] ** 2
    return f

In [1621]:
def Jacobian(x):
  return np.array([
      [2*x[0] - 16.42, 2*x[1],         -2*x[2]],
      [2*x[0] - 0.68,  2*x[1] - 13.24, -2*x[2]],
      [2*x[0] - 11.92, 2*x[1] + 2.24,  -2*x[2]]
  ])

### b) Resolva o sistema não-linear utilizando os códigos feitos nos itens 2 e 3

In [1622]:
# old algorithm
error = 1e-10
initial_guess = np.array([1, 1, 1])
a2, b2, R2 = solve_newton_system_with_absolute_error(Function, Jacobian, initial_guess, error)
print(f'a = {np.round(a2, 3)}, b = {np.round(b2, 3)}, R = {np.round(R2, 3)}')

a = 4.83, b = 3.97, R = 5.214


In [1623]:
# new algorithm

a3, b3, R3 = newton_new_algorithm(Function, Jacobian, initial_guess, error)
print(f'a = {a3}, b = {b3}, R = {R3}')

a = 4.830105654297453, b = 3.9699216766345855, R = 5.213824307236372


### c) Houve melhora no tempo de execução do código implementado no item 3?

In [1624]:

before = time.time()
solve_newton_system_with_absolute_error(Function, Jacobian, initial_guess, 10e-10)
after = time.time()
print(f'Old algorithm: {after-before} s.')

before = time.time()
newton_new_algorithm(Function, Jacobian, [1, 1, 1], 10e-10)
after = time.time()
print(f'New algorithm: {after - before} s.')

Old algorithm: 0.0010411739349365234 s.
New algorithm: 0.00037384033203125 s.
